<h1 align="center">Sentiment Analysis with LSTMs</h1>

First, we need to create word vectors. For simplicity, we will use a pretrained Word2Vec model
with Ukrainian words-vectors, each of which has a dimension of 300.

This sample of word vectors was created on the basis of fiction literature. We chose the lematized version of this model because we already have our sample, which we processed in the previous part, which would fit perfectly here.

The model can be found on [this website](https://lang.org.ua/uk/models/).

To get started, let's download the necessary libraries:

In [1]:
import numpy as np
import pandas as pd
import pickle
import gensim, logging
import gensim.models.keyedvectors as word2vec
import matplotlib.pyplot as plt

%matplotlib inline

Next, we will load the sample data we processed in the previous part:

In [2]:
with open('documents.pql', 'rb') as f:
     docs = pickle.load(f)

In [3]:
print(docs[2])

(['насторожитися', 'назва', 'взагалі', 'стосуватися', 'збільшення', 'втрата', 'вага', 'вага', 'випадковий', 'запекло', 'розумний', 'роман', 'дорота', 'відчайдушно', 'самотній', 'жінка', 'обидва', 'жертва', 'жорстокий', 'поводження', 'різний', 'батьки', 'підводити', 'дитинство', 'сповнений', 'сором', 'самодиверсія', 'писати', 'складно', 'усвідомлюючи', 'жертва', 'жертва', 'сентиментальний', 'розповідь', 'біль', 'сексуальний', 'сцена', 'любити', 'кінець', 'розчарування', 'самотність', 'лежати', 'бурхливий', 'гнів', 'нездатність', 'люди', 'язуватися', 'раса', 'клас', 'переважно', 'стать', 'відчути', 'анна', 'граніт', 'інтелігентка', 'тінь', 'домінувати', 'наратив', 'дорота', 'джастін', 'вперше', 'зустрічатися', 'платформа', 'проява', 'гострий', 'інтелект', 'натяк', 'дидактизм', 'незначний', 'каверза', 'останній', 'розділ', 'зворушити', 'довести', 'огидний', 'сентиментальність', 'безумовно', 'справляти', 'настрій', 'людство', 'слово', 'зустрічатися', 'книга', 'сексуальність', 'язаний', 'лю

In [4]:
print("Number of documents:", len(docs))

Number of documents: 2000


Now we will load our word2vec model. 

This may take some time, as the model contains 325 250  words, so we will get a 325 250 x 300 embedding matrix that contains all the values of the word vectors.

In [5]:
model = word2vec.KeyedVectors.load_word2vec_format('ubercorpus.lowercased.lemmatized.word2vec.300d.txt', binary=False)

Now let's get a list of all the words from our dictionary:

In [6]:
words = list(model.vocab)

Just to make sure everything is loaded correctly, we can look at the dimensions of the dictionary list and the embedding matrix:

In [7]:
print(words[:50], "\n\nTotal words:", len(words), "\n\nWord-Vectors shape:", model.vectors.shape)

['в', 'на', 'у', 'і', 'з', 'що', 'не', 'бути', 'до', 'за', 'та', 'це', 'україна', 'який', 'як', 'про', 'рік', 'а', 'він', 'для', 'від', 'свій', 'вони', 'його', 'цей', 'також', 'я', 'ми', 'той', 'й', 'категорія', 'один', 'мати', 'час', 'такий', 'із', 'але', 'могти', 'під', 'рок', 'президент', 'по', 'те', 'вона', 'після', 'сказати', 'країна', 'український', 'інший', 'так', 'ще', 'заявити', 'область', '1', 'слово', 'її', 'росія', 'вже', 'голова', 'всі', 'повідомити', 'при', 'або', 'тому', 'справа', 'київ', 'через', 'місто', 'щоб', 'особа', 'чи', 'питання', 'люди', 'місце', 'державний', 'партія', 'рада', 'все', 'новий', 'якщо', 'наш', 'влада', 'повідомляти', 'рішення', 'сьогодні', 'район', 'українська', 'депутат', '2', 'ж', 'щодо', 'де', 'міністр', 'національний', 'сам', 'уніан', 'лише', 'себе', 'м', 'булий'] 

Total words: 325250 

Word-Vectors shape: (325250, 300)


We can also find a word like "гарний" in our word list and then access the corresponding vector through the embedding matrix:

In [8]:
print(model['гарний'])

[ 4.516500e-02  1.251880e-01  2.108905e+00  1.637814e+00 -4.077700e-02
  7.728100e-01 -6.516440e-01  1.463697e+00  3.339670e-01 -6.916780e-01
 -1.779990e-01  4.337260e-01  7.660550e-01  9.300980e-01 -1.599930e-01
  2.290509e+00 -5.549170e-01  6.714390e-01 -4.648900e-02 -2.138202e+00
  8.052800e-02  7.673460e-01 -2.694449e+00 -2.138186e+00  6.990550e-01
 -2.908490e-01 -4.759980e-01 -1.400164e+00 -1.804889e+00 -7.014440e-01
 -1.354321e+00 -4.605037e+00 -2.812303e+00  1.006020e+00 -4.980150e-01
  7.537600e-01 -2.421496e+00  2.177093e+00 -1.282774e+00  2.158463e+00
  5.019600e-01 -1.288280e-01 -1.434353e+00  1.179314e+00 -4.570560e-01
  1.129010e-01  3.679680e-01  1.147463e+00 -2.718560e-01 -4.053980e-01
 -1.606960e+00 -1.048291e+00 -1.496587e+00  1.050120e-01 -4.461910e-01
 -6.268950e-01  1.559451e+00  4.125400e-02 -5.025610e-01 -4.405390e-01
  5.498720e-01  6.160980e-01 -1.434053e+00 -3.693590e-01  9.504800e-01
  5.033900e-02  6.530420e-01  1.092300e-01  1.099505e+00 -5.173930e-01
  4.52

<h2 align="center">Word Average Embedding Model</h2>

Well, let's start analyzing our vectors. Our first approach will be the **word average embedding model**. 

The essence of this naive approach is to take the average of all word vectors from a sentence to get one 300-dimensional vector that represents the tone of the whole sentence that we feed the model and try to get some quick result.

We didn't have to put a try/except, but even though I cleaned up our sample, there were a couple of hundred words left after the processing that needed to be searched for and removed, such as `жорда`, `дарба`, `еріот`, `енрон`. These are not even words, so their absence will not affect anything.

In [16]:
def sent_embed(words, docs):
    x_sent_embed, y_sent_embed = [], []
    count_words, count_non_words = 0, 0  
    
    # recover the embedding of each sentence with the average of the vector that composes it
    # sent - sentence, state - state of the sentence (pos/neg)
    for sent, state in docs:
        # average embedding of all words in a sentence
        sent_embed = []
        for word in sent:
            try:
                count_words += 1
                sent_embed.append(model[word])
            except KeyError:
                count_non_words += 1
                sent_embed.append(0)
        
        # add a sentence vector to the list
        x_sent_embed.append(np.mean(sent_embed, axis=0).tolist())
        
        # add a label to y_sent_embed
        if state == 'pos': y_sent_embed.append(1)
        elif state == 'neg': y_sent_embed.append(0)
            
    print(count_non_words, "out of", count_words, "words were not found in the vocabulary.")
    
    return x_sent_embed, y_sent_embed

In [17]:
x, y = sent_embed(words, docs)

3057 out of 143957 words were not found in the vocabulary.


<h2 align="center">Cosine Similarity</h2>

To measure the similarity of 2 words, we need a way to measure the degree of similarity between 2 embedding vectors for these 2 words. Given 2 vectors $u$ and $v$, cosine similarity is determined as follows:

$$\text{cosine_similarity(u, v)} = \frac {u . v} {||u||_2 ||v||_2} = cos(\theta)$$

where: 

* $u.v$ - dot product (or inner product) of two vectors;

* $||u||_2$ - norm (or length) of the vector $u$;
    
    * **Note**: norm of $u$ is defined as $ ||u||_2 = \sqrt{\sum_{i=1}^{n} u_i^2}$)

* $\theta$ is the angle between $u$ and $v$. 

This similarity depends on the angle between $u$ and $v$. If $u$ and $v$ are very similar, their cosine similarity will be close to 1; if they are dissimilar, the cosine similarity will take a smaller value. 

**`cosine_similarity()`** is a method that used to estimate the similarity between word vectors.

In [20]:
def cosine_similarity(u, v):
    """
    Cosine similarity reflects the degree of similariy between u and v
        
    Arguments:
        u -- a word vector of shape (n,)          
        v -- a word vector of shape (n,)

    Returns:
        cosine_similarity -- the cosine similarity between u and v defined by the formula above.
    """
    
    distance = 0.0
    
    # compute the dot product between u and v
    dot = np.dot(u,v)
    
    # compute the L2 norm of u
    norm_u = np.sqrt(sum(u**2))
    
    # Compute the L2 norm of v
    norm_v = np.sqrt(sum(v**2))
    
    # Compute the cosine similarity defined by formula above
    cosine_similarity = dot/(norm_u*norm_v)
    
    return cosine_similarity

Let's check the cosine similarity on 2 negative sentences:

In [71]:
print("Sentence #1: ", docs[1], "\n\nSentence #5: ", docs[5])
print("\nSentence Embedding #1: ", x[1], "\n\nSentence Embedding #5: ", x[5])

Sentence #2:  (['книга', 'читатися', 'неглибокий', 'довгий', 'стаття', 'магазін', 'магазин', 'схожий', 'спеціалізуватися', 'книга', 'самодопомога', 'навряд', 'існувати', 'літературний', 'відчуття', 'стиль', 'дратувати', 'наприклад', 'писати', 'тато', 'любити', 'книга', 'тоді', 'очікувати', 'більший', 'лексика'], 'neg') 

Sentence #18:  (['серія', 'пародія', 'розуміти', 'відчуття', 'втрата', 'закінчувати', 'улюблений', 'серіал', 'мара', 'макбрайда', 'мати', 'останній', 'слово', 'досвід', 'родина', 'стільки', 'книга', 'звучати', 'неправдиво', 'незграбний', 'історичний', 'натяк', 'оповідальний', 'голос', 'мізерний', 'гумор', 'один', 'найкращий', 'робот', 'лорати', 'уайлдерти', 'голос', 'прогресувати', 'зрілість', 'головний', 'героїня', 'ріст', 'великий', 'ліса', 'написаний', 'маленький', 'дівчатко', 'щасливий', 'золото', 'молодий', 'жінка', 'справлятися', 'розмовляти', 'читач', 'книга', 'крихітний', 'нитка', 'лорати', 'недостатній', 'зробити', 'розчарувати', 'шанувальник', 'вибірка', 'при

In [70]:
print("cosine_similarity = ", cosine_similarity(np.array(x[1]), np.array(x[5])))

cosine_similarity =  0.7370330086343598


A value of 0.73 indicates that the sentences are close to each other, and so it is.

Let's check on two positive sentences:

In [97]:
print("Sentence #10: ", docs[10], "\n\nSentence #12: ", docs[12])
print("\nSentence Embedding #10: ", x[10], "\n\nSentence Embedding #12: ", x[12])

Sentence #10:  (['помилятися', 'гаразд', 'визнати', 'думати', 'науковий', 'фантастика', 'роман', 'футуристичний', 'чашка', 'читаючи', 'змова', 'смерть', 'зрозуміти', 'помилятися', 'любити', 'футуристичний', 'серія', 'бездомний', 'вбивати', 'справа', 'передавати', 'даллас', 'випадковий', 'насильство', 'чоловік', 'снукс', 'померти', 'серце', 'хірургічно', 'видалений', 'видаватися', 'досвідчений', 'хірург', 'коли', 'даллас', 'копатися', 'виявляти', 'злочин', 'язанити', 'скоєний', 'йорк', 'похилий', 'ліцензований', 'супутник', 'померти', 'результат', 'видалення', 'печінка', 'після', 'більший', 'копання', 'даллас', 'виявляти', 'випадок', 'чикаго', 'кордон', 'подібний', 'результат', 'продаж', 'чорне', 'ринок', 'орган', 'жертва', 'померти', 'місяць', 'єва', 'даллас', 'знайти', 'вбивець', 'довестися', 'ясуват', 'обраний', 'жертва', 'ближче', 'наближатися', 'відповідь', 'нервуватися', 'вплив', 'політик', 'лікар', 'коли', 'наблизитися', 'істина', 'лиходіяти', 'повинний', 'припинити', 'розслідува

In [98]:
print("cosine_similarity = ", cosine_similarity(np.array(x[10]), np.array(x[12])))

cosine_similarity =  0.6527750583474209


These sentences are also close to each other. 

So now let's check sentences with different states:

In [157]:
print("Sentence #0: ", docs[7], "\n\nSentence #12: ", docs[13])

Sentence #0:  (['каліграф', 'шукати', 'вірш', 'молитва', 'написати', 'написати', 'рамка', 'подарувати', 'молодята', 'схвильований', 'відкрити', 'книга', 'прочитати', 'прекрасний', 'вірш', 'казати', 'небо', 'каліграф', 'брат', 'міністр', 'наближення', 'сезон', 'весілля', 'замовляти', 'копія', 'якби', 'оцінити', 'книга', 'зірка', 'дякувати'], 'pos') 

Sentence #12:  (['інформація', 'міститися', 'книга', 'надзвичайно', 'застарілий', 'сильне', 'акцент', 'стероїд', 'шкода', 'власниця', 'домашній', 'тварина', 'актуальний', 'корисний', 'книга', 'широко', 'розповсюджений', 'проблема'], 'neg')


In [156]:
print("cosine_similarity = ", cosine_similarity(np.array(x[7]), np.array(x[13])))

cosine_similarity =  0.21123042365281994


Here the similarity is extremely small, which means that the sentences are not similar to each other.